In [126]:
import azureml.core
from azureml.core import Workspace, Run, Experiment, Dataset
from azureml.core.compute import ComputeTarget, DataFactoryCompute
from azureml.core.datastore import Datastore
from azureml.data.data_reference import DataReference
from azureml.exceptions import ComputeTargetException
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import AdlaStep, AzureBatchStep, DataTransferStep
from azureml.pipeline.steps_internal import ScopeStep

In [127]:
subscription_id="4aaa645c-5ae2-4ae9-a17a-84b9023bc56a"
resource_group="itp-pilot-ResGrp"
name="itp-pilot"

In [128]:
from azureml.core import Workspace

ws = Workspace.get(subscription_id=subscription_id, resource_group=resource_group, name=name)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

itp-pilot
itp-pilot-ResGrp
westeurope
4aaa645c-5ae2-4ae9-a17a-84b9023bc56a


In [129]:
adl_datastore_name = 'searchparner_test'
adls_datastore = Datastore.get(ws, adl_datastore_name)
print("found datastore with name: %s" % adl_datastore_name)

""" test data reference
input_data = DataReference(
    adls_datastore, data_reference_name='RawData', path_on_datastore='/local/temp/chjinche/large_data/bing_data_label.tsv')
"""


input_data = ws.datasets['BotDetectionLabelData'].as_named_input('RawData')


found datastore with name: searchparner_test


In [130]:
script_folder = './script'

In [131]:
# convert tsv to ss

ss_data = PipelineData('SS_Data', datastore=adls_datastore)

step_ssconvert_params = {"ExtractClause": "Label:string, Id:string"}

step_ssconvert = ScopeStep(
    name = 'Convert Tsv to SS',
    script_name = 'convert2ss.script',
    custom_job_name_suffix='_bing_bot_data',
    inputs = [input_data],
    outputs = [ss_data],
    params = step_ssconvert_params,
    scope_param = "-tokens 30",
    adla_account_name = 'searchdm-partner-c09',
    allow_reuse = True,
    source_directory = script_folder
)

In [132]:
# convert ss to tsv

tsv_data = PipelineData('Tsv_Data', datastore=adls_datastore)

step_tsvconvert = ScopeStep(
    name = 'Convert SS to Tsv',
    script_name = 'convert2tsv.script',
    custom_job_name_suffix='_bing_bot_data',
    inputs = [ss_data],
    outputs = [tsv_data],
    scope_param = "-tokens 30",
    adla_account_name = 'searchdm-partner-c09',
    allow_reuse = True,
    source_directory = script_folder
)

In [133]:
my_pipeline = Pipeline(
    description = 'single step pipeline to submit scope job',
    workspace = ws,
    steps = [step_tsvconvert]
)

In [134]:
from azureml.core import Experiment

exp = Experiment(ws, 'scope_experiment')
pipeline_run = exp.submit(my_pipeline)

Created step Convert SS to Tsv [d67f570c][e0c2e0d4-7eb5-432a-972d-4a6cf12ea2e7], (This step is eligible to reuse a previous run's output)
Created step Convert Tsv to SS [53d4769f][78566538-e940-4b24-b4f8-45e96c028d23], (This step will run and generate new outputs)
Submitted PipelineRun 38f5e60a-aad9-4977-8f59-dde1f1734d1a
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/scope_experiment/runs/38f5e60a-aad9-4977-8f59-dde1f1734d1a?wsid=/subscriptions/4aaa645c-5ae2-4ae9-a17a-84b9023bc56a/resourcegroups/itp-pilot-ResGrp/workspaces/itp-pilot
